In [2]:
#성능을 높일 수 있는 방법 소개

import numpy as np
from google.colab import drive

#load train data and label
drive.mount('/content/drive')
train_data_path = "/content/drive/My Drive/ai_data/train.npy" # Your train data file
train_label_path = "/content/drive/My Drive/ai_data/train_label.npy" # Your train label file
test_query_path = "/content/drive/My Drive/ai_data/test_query.npy" # Your train data file
test_gallery_path = "/content/drive/My Drive/ai_data/test_gallery.npy" # Your train label file

test_query_label_path = "/content/drive/My Drive/ai_data/test_query_label.npy" # 
test_gallery_label_path = "/content/drive/My Drive/ai_data/test_gallery_label.npy" # 


train_data = np.load(train_data_path, allow_pickle=True)
train_label = np.load(train_label_path, allow_pickle=True)  

#test_query_label = np.load(test_query_label_path, allow_pickle=True)

#쿼리는 질문(검색을 예로 들면 이미지 검색시 올리는 이미지)
test_gallery_label = np.load(test_gallery_label_path, allow_pickle=True)  
#데이터베이스에 저장되어있는 것들이 갤러리(찾고싶은 모든 이미지들이 저장된 곳)
#쿼리 1개를 전체 갤러리중 가장 가까운 하나를 찾는
#쿼리가 1개씩 들어올 때 가장 가까운 것을 갤러리에서 리턴해준다.
test_query_data = np.load(test_query_path, allow_pickle=True)
test_gallery_data = np.load(test_gallery_path, allow_pickle=True)

num_train_data = train_data.shape[0]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
print(train_data.shape) # train_data is a list
print(num_train_data) # the number of train_data
print(train_label.shape)

(2000, 1, 512, 1, 1)
2000
(2000,)


In [6]:
def getNearestNeibor(query, gallery): #쿼리 데이터와 갤러리 데이터를 비교하여 어떤 것과 가장 가까운지 인덱스 반환
  num_query = query.shape[0]
  num_gallery = gallery.shape[0]
  nn_idx = np.zeros(num_query)
  for q in range(num_query):  #2중 포문
    dist = np.zeros(num_gallery)
    for g in range(num_gallery):
      dist[g] = np.sqrt(np.sum((query[q,:] - gallery[g,:]) ** 2, axis=0)) #q번째 쿼리, g번째 갤러리 인덱싱 하여 하나하나 비교
      #axis를 통해 차원설정, distance 연산 dist의 갯수는 gallery의 수와 같다.
    nn_idx[q] = np.argmin(dist)
    #query하나를 모든 gallery와 비교하고 그 distance중 가장 작은 것을 저장
  return nn_idx

In [4]:
import numpy as np
a=[1,2,3,4,5]
b=[1,3,5,7,9]
c=np.array(a)-np.array(b) #리스트 연산을 하기 위해 numpy로
print(c)
print(c**2)
print(np.sum(c**2))
print(np.sqrt((np.sum(c**2))))

[ 0 -1 -2 -3 -4]
[ 0  1  4  9 16]
30
5.477225575051661


In [7]:
nn_idx = getNearestNeibor(test_query_data.squeeze(), test_gallery_data.squeeze())
print(test_query_data.shape)
print(test_query_data.squeeze().shape) #squeeze는 1을 모두 없앰(쓸모없는 것들)

(1000, 1, 512, 1, 1)
(1000, 512)


In [9]:
nn_idx = getNearestNeibor(test_query_data.squeeze(), test_gallery_data.squeeze())
print(test_query_data.shape)
#print(np.sum(test_query_label == test_gallery_label[np.int64(nn_idx)]))
#label끼리 비교하여 얼마나 정확한지 척도를 확인함
#nn_idx_rand = np.random.randint(1,100,1000)
#print(np.sum(test_query_label == test_gallery_label[np.int64(nn_idx_rand)]))

(1000, 1, 512, 1, 1)


In [10]:
pred_test_gallery_label_txt = list_data = [str(int(a)).strip('\n\r') for a in test_gallery_label[np.int64(nn_idx)]]
print(pred_test_gallery_label_txt)
#print(train_data[0:10])

['170', '200', '116', '92', '116', '116', '116', '56', '187', '116', '172', '88', '170', '32', '90', '116', '182', '158', '116', '36', '122', '60', '122', '188', '14', '116', '92', '86', '23', '71', '122', '122', '168', '103', '33', '18', '172', '45', '158', '14', '170', '71', '71', '146', '170', '86', '33', '103', '122', '196', '130', '116', '46', '116', '83', '170', '122', '182', '92', '30', '19', '182', '60', '71', '122', '116', '158', '14', '71', '188', '170', '116', '68', '188', '146', '158', '182', '116', '116', '122', '170', '116', '75', '116', '23', '14', '122', '116', '14', '116', '35', '32', '131', '116', '116', '170', '116', '92', '116', '122', '200', '77', '33', '122', '40', '71', '116', '158', '116', '122', '116', '170', '182', '33', '158', '40', '200', '116', '88', '116', '40', '116', '170', '122', '106', '116', '132', '185', '88', '62', '116', '116', '170', '19', '28', '170', '179', '116', '170', '14', '116', '157', '116', '157', '19', '14', '122', '122', '155', '116', '

In [11]:
#Import Libraries
#여기서 부터 딥러닝을 통해 인공지능 학습

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [14]:
class MyDataset(torch.utils.data.Dataset):
  #재정의
  def __init__(self, data, label=[]): #데이터와 레이블
    self.data = data
    self.label = label
    
  def __len__(self):  #데이터의 길이
    return self.data.shape[0]

  def __getitem__(self, idx): #어떤걸 어떻게 가져올지
    if self.label == []:
      return self.data[idx, :, :, :]
    else:
      return self.data[idx, :, :, :], self.label[idx]


In [17]:
args={}
kwargs={}
args['batch_size']=8
args['test_batch_size']=16
args['epochs']=50  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.1 #Learning rate is how fast it will decend. 얼만큼 많이 학습 될 것인지, 배치사이즈 증가시 러닝레이트 증가
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).
#하산시 가속도를 얼마나 유지할 지
args['seed']=1 #random seed
args['log_interval']=1000 #프린트를 얼마나 자주 할 것인지, 값이 작을 수록 세밀하게 프린트
args['cuda']=False



class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(512, 512, kernel_size=1) #인풋 사이즈, 아웃풋 사이즈
        self.bn1 = nn.BatchNorm2d(512) #입력 사이즈(정규화 해주는 레이어)
        self.conv2 = nn.Conv2d(512, 200, kernel_size=1)
        #self.conv2_drop = nn.Dropout2d()  #Dropout
        #self.fc1 = nn.Linear(512, 64)
        self.fc2 = nn.Linear(512, 200)

    def forward(self, x):
        #Convolutional Layer/Pooling Layer/Activation
        x = F.relu( self.bn1(self.conv1(x)), 2) #non_linearity를 위해 ReLU
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        #x = F.relu(self.conv2_drop(self.conv2(x)), 2)
        x = x.view(-1, 512) # 1들을 제거해줌 1,1,512,100 -> 512,100
        #Fully Connected Layer/Activation
        #x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x) #prediction을 위해 512 -> 200으로
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1) #확률의 합이 1이 되도록




def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader): #배치 사이즈 만큼 가져옴
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable.
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data) #포워드 함수 실행(아웃풋은 최종 결과 값인 확률 값)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target) #타겟은 정답인 label
        #dloss/dx for every Variable 
        loss.backward() #뉴럴 네트워크를 얼만큼 업데이트 해야하는가
        #print(loss.data)
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum() #정답 갯수

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [18]:
train_data_reshaped1 = np.reshape(train_data[ :-1:2,:,:,:,:],(train_data.shape[0]//2, train_data.shape[2], train_data.shape[3], train_data.shape[4]))
train_label = np.squeeze(train_label)
train_loader = torch.utils.data.DataLoader( #데이터 로더는 데이터셋, 배치사이즈, 셔플을 이용해서 더 많은 기능을 제공
    MyDataset(torch.from_numpy(train_data_reshaped1), torch.squeeze(torch.from_numpy(train_label[:-1:2]))), #마이 데이터셋 객체화(데이터만 저장 된 객체)
    batch_size=args['batch_size'], shuffle=True, **kwargs)
train_data_reshaped2 = np.reshape(train_data[1:-1:2,:,:,:,:],(train_data.shape[0]//2-1, train_data.shape[2], train_data.shape[3], train_data.shape[4]))
test_loader = torch.utils.data.DataLoader( #트레인데이터를 다르게 했으므로 정확도가 100으로 안나온다
    MyDataset(torch.from_numpy(train_data_reshaped2), torch.squeeze(torch.from_numpy(train_label[1:-1:2]))),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

In [19]:
model = Net()
if args['cuda']:
    model.cuda() #기본적으론 CPU에 올라가지만 이를 통해 GPU에 올릴 수도 있다

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])
#어떻게 효과적으로 내려올 것인가?, loss func의 기울기를 보고 내려오는 방식
for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/1000 (0%)]	Loss: 5.361429


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 18.0017, Accuracy: 8/999 (1%)

Train Epoch: 2 [0/1000 (0%)]	Loss: 2.360949

Test set: Average loss: 15.0944, Accuracy: 19/999 (2%)

Train Epoch: 3 [0/1000 (0%)]	Loss: 0.144964

Test set: Average loss: 6.7963, Accuracy: 42/999 (4%)

Train Epoch: 4 [0/1000 (0%)]	Loss: 0.050696

Test set: Average loss: 4.4117, Accuracy: 102/999 (10%)

Train Epoch: 5 [0/1000 (0%)]	Loss: 0.030618

Test set: Average loss: 4.5001, Accuracy: 106/999 (11%)

Train Epoch: 6 [0/1000 (0%)]	Loss: 0.012054

Test set: Average loss: 4.4235, Accuracy: 107/999 (11%)

Train Epoch: 7 [0/1000 (0%)]	Loss: 0.012339

Test set: Average loss: 4.5431, Accuracy: 92/999 (9%)

Train Epoch: 8 [0/1000 (0%)]	Loss: 0.004836

Test set: Average loss: 4.4762, Accuracy: 108/999 (11%)

Train Epoch: 9 [0/1000 (0%)]	Loss: 0.073079

Test set: Average loss: 4.5241, Accuracy: 111/999 (11%)

Train Epoch: 10 [0/1000 (0%)]	Loss: 0.031347

Test set: Average loss: 4.4912, Accuracy: 108/999 (11%)

Train Epoch: 11 [0/1000 (0%)]	

In [20]:
test_query_data_reshaped = np.reshape(test_query_data,(test_query_data.shape[0], test_query_data.shape[2], test_query_data.shape[3], test_query_data.shape[4]))
test_gallery_data_reshaped = np.reshape(test_gallery_data,(test_gallery_data.shape[0], test_gallery_data.shape[2], test_gallery_data.shape[3], test_gallery_data.shape[4]))

test_query_loader = torch.utils.data.DataLoader(
    MyDataset(torch.from_numpy(test_query_data_reshaped)),
    batch_size=args['test_batch_size'], shuffle=False, **kwargs)
test_gallery_loader = torch.utils.data.DataLoader(
    MyDataset(torch.from_numpy(test_gallery_data_reshaped)),
    batch_size=args['test_batch_size'], shuffle=False, **kwargs)

In [21]:
import math
feature_extractor = torch.nn.Sequential(*list(model.children())[:-2])

test_query_feature = np.zeros((test_query_data.shape[0], 512)) #conv1 -> batchnorm -> relu통과한 것
test_gallery_feature = np.zeros((test_gallery_data.shape[0], 512))

cursor = 0
for data in test_query_loader:
  test_query_feature[cursor:min(cursor+args['test_batch_size'], test_query_data.shape[0]) ,:] = feature_extractor(data).detach().numpy().squeeze()
  cursor+=args['test_batch_size']

cursor = 0
for data in test_gallery_loader:
  test_gallery_feature[cursor:min(cursor+args['test_batch_size'], test_gallery_data.shape[0]) ,:] = feature_extractor(data).detach().numpy().squeeze()
  cursor+=args['test_batch_size']


In [22]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

In [25]:
print(test_query_feature.shape)
print(test_gallery_feature.shape)
nn_idx_deep = getNearestNeibor(normalized(test_query_feature, axis=1, order=2), normalized(test_gallery_feature, axis=1, order=2))
#normalization을 통해 성능이 좋아진다. 피쳐들이 나타내는 정보들의 성능이 좋아지기 때문에 왠만해선 성능이 높아진다.
#nn_idx_deep = getNearestNeibor(test_query_feature, test_gallery_feature)
print(test_query_data.shape)
#print(np.sum(test_query_label == test_gallery_label[np.int64(nn_idx_deep)]))
#print(np.sum(test_query_label == test_gallery_label[np.int64(nn_idx)]))
#nn_idx_rand = np.random.randint(1,100,1000)
#print(np.sum(test_query_label == test_gallery_label[np.int64(nn_idx_rand)]))

(1000, 512)
(100, 512)
(1000, 1, 512, 1, 1)


In [27]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1).fit(normalized(test_gallery_feature, axis=1, order=25))
distances, indices = nbrs.kneighbors(normalized(test_query_feature, axis=1, order=25))
#오더가 높아질 수록 성능 내려가는 경향 l1, l2여야 성능이 좋은 경향을 보인다. 보통 l2가 많이 쓰인다.
#print(np.sum(test_query_label == test_gallery_label[np.int64(indices.squeeze())]))


# 새 섹션